In [1]:
# Код для человека

import os
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderUnavailable
import pytz
from datetime import datetime
import swisseph as swe
import json
from math import floor

class NatalChartCalculator:
    def __init__(self):
        # Инициализация Swiss Ephemeris
        current_dir = os.getcwd() #os.path.dirname(os.path.abspath(__file__))
        ephe_path = os.path.join(current_dir, 'ephe')
        swe.set_ephe_path(ephe_path)
        swe.set_jpl_file('de441.eph')  # Используем точные эфемериды DE441

        # Настройки точности
        self.ORBS = {
            'major': {'Sun': 8, 'Moon': 8, 'Planets': 6, 'Axis': 5},
            'minor': {'Sun': 5, 'Moon': 5, 'Planets': 4, 'Axis': 3}
        }

        # Системы домов (автоматический выбор по широте)
        self.house_systems = {
            'P': 'Placidus',
            'K': 'Koch',
            'R': 'Regiomontanus'
        }

        # Планеты и важные точки
        self.planets = {
            swe.SUN: 'Sun',
            swe.MOON: 'Moon',
            swe.MERCURY: 'Mercury',
            swe.VENUS: 'Venus',
            swe.MARS: 'Mars',
            swe.JUPITER: 'Jupiter',
            swe.SATURN: 'Saturn',
            swe.URANUS: 'Uranus',
            swe.NEPTUNE: 'Neptune',
            swe.PLUTO: 'Pluto',
            swe.CHIRON: 'Chiron',
            swe.TRUE_NODE: 'North Node',
            swe.OSCU_APOG: 'South Node'
        }

        self.zodiac_signs = [
            "Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo",
            "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"
        ]

        # Аспекты и их орбисы
        self.aspects = {
            0: ('Conjunction', 'major'),
            30: ('Semi-Sextile', 'minor'),
            45: ('Semi-Square', 'minor'),
            60: ('Sextile', 'major'),
            72: ('Quintile', 'minor'),
            90: ('Square', 'major'),
            120: ('Trine', 'major'),
            135: ('Sesqui-Square', 'minor'),
            144: ('Biquintile', 'minor'),
            150: ('Quincunx', 'minor'),
            180: ('Opposition', 'major')
        }

    def get_city_coordinates(self, city_name):
        """Получение точных координат с резервными значениями"""
        try:
            geolocator = Nominatim(user_agent="astro_app_v1.2")
            location = geolocator.geocode(city_name, exactly_one=True)
            if location:
                return (location.latitude, 
                        location.longitude, 
                        location.altitude if location.altitude else 0.0)
        except Exception:
            pass
            
        # Резервные координаты для России
        known_cities = {
            "москва": (55.7558, 37.6173, 156),
            "санкт-петербург": (59.9343, 30.3351, 3),
            "новосибирск": (55.0084, 82.9357, 177),
            "екатеринбург": (56.8389, 60.6057, 237),
            "калининград": (54.7104, 20.4522, 25),
            "мга": (59.7569, 31.0609, 33)
        }
        city_lower = city_name.strip().lower()
        for name, coords in known_cities.items():
            if name in city_lower:
                return coords
        return (55.7558, 37.6173, 156)  # Москва по умолчанию

    def calculate_planet_positions(self, jd_ut):
        """Точный расчет положений планет с учетом скорости"""
        positions = {}
        for planet_id, name in self.planets.items():
            flags = swe.FLG_SWIEPH | swe.FLG_SPEED
            if planet_id in [swe.TRUE_NODE, swe.OSCU_APOG]:
                flags |= swe.FLG_TRUEPOS
            
            pos, ret_flags = swe.calc_ut(jd_ut, planet_id, flags)
            
            lon = pos[0] % 360
            positions[name] = {
                'longitude': lon,
                'latitude': pos[1],
                'distance': pos[2],
                'speed': pos[3],
                'retrograde': pos[3] < 0,
                'sign': self.zodiac_signs[floor(lon / 30)],
                'position_in_sign': lon % 30,
                'sign_emoji': self.get_sign_emoji(lon)
            }
        return positions

    def get_house_system(self, lat):
        """Автоматический выбор оптимальной системы домов по широте"""
        abs_lat = abs(lat)
        if abs_lat > 60:
            return b'K'  # Koch для высоких широт
        elif abs_lat > 55:
            return b'R'  # Regiomontanus для средних широт
        return b'P'      # Placidus для низких широт

    def calculate_houses(self, jd_ut, lat, lon):
        """Расчет домов с автоматическим выбором системы"""
        hsys = self.get_house_system(lat)
        cusps, ascmc = swe.houses(jd_ut, lat, lon, hsys)
        
        houses = {}
        for i, cusp in enumerate(cusps[:12]):
            cusp_deg = cusp % 360
            houses[f'House_{i+1}'] = {
                'cusp': cusp_deg,
                'sign': self.zodiac_signs[floor(cusp_deg / 30)],
                'position_in_sign': cusp_deg % 30,
                'sign_emoji': self.get_sign_emoji(cusp_deg)
            }
        
        return {
            'houses': houses,
            'ascendant': ascmc[0] % 360,
            'midheaven': ascmc[1] % 360,
            'system': self.house_systems.get(hsys.decode(), 'Placidus')
        }

    def calculate_aspects(self, planets, asc, mc):
        """Расчет всех аспектов с учетом углов"""
        aspects = []
        points = {**planets, 'Ascendant': {'longitude': asc}, 'Midheaven': {'longitude': mc}}
        point_names = list(points.keys())
        
        for i in range(len(point_names)):
            for j in range(i+1, len(point_names)):
                p1 = point_names[i]
                p2 = point_names[j]
                lon1 = points[p1]['longitude']
                lon2 = points[p2]['longitude']
                
                angle = abs(lon1 - lon2)
                angle = min(angle, 360 - angle)
                
                for aspect_angle, (aspect_name, aspect_type) in self.aspects.items():
                    orb_config = self.ORBS[aspect_type]
                    if p1 in ['Ascendant', 'Midheaven'] or p2 in ['Ascendant', 'Midheaven']:
                        orb = orb_config['Axis']
                    elif p1 == 'Sun' or p2 == 'Sun':
                        orb = orb_config['Sun']
                    elif p1 == 'Moon' or p2 == 'Moon':
                        orb = orb_config['Moon']
                    else:
                        orb = orb_config['Planets']
                    
                    if abs(angle - aspect_angle) <= orb:
                        aspects.append({
                            'planet1': p1,
                            'planet2': p2,
                            'aspect': aspect_name,
                            'angle': angle,
                            'exact_angle': aspect_angle,
                            'orb': abs(angle - aspect_angle),
                            'aspect_type': aspect_type
                        })
        
        # Сортируем аспекты по важности (мажорные сначала) и орбису
        aspects.sort(key=lambda x: (x['aspect_type'] != 'major', x['orb']))
        return aspects

    def get_sign_emoji(self, longitude):
        """Возвращает emoji для знака зодиака"""
        sign_index = floor(longitude / 30)
        emojis = ['♈', '♉', '♊', '♋', '♌', '♍', '♎', '♏', '♐', '♑', '♒', '♓']
        return emojis[sign_index]

    def calculate_natal_chart(self, city_name, birth_datetime_local, timezone_str):
        """Основной расчет натальной карты"""
        # Получение координат
        lat, lon, elevation = self.get_city_coordinates(city_name)
        
        # Преобразование времени
        local_tz = pytz.timezone(timezone_str)
        birth_local = local_tz.localize(birth_datetime_local)
        birth_utc = birth_local.astimezone(pytz.utc)
        
        # Юлианская дата с точностью до секунды
        jd_ut = swe.julday(
            birth_utc.year,
            birth_utc.month,
            birth_utc.day,
            birth_utc.hour + birth_utc.minute/60 + birth_utc.second/3600
        )
        
        # Расчет основных данных
        planets = self.calculate_planet_positions(jd_ut)
        houses_data = self.calculate_houses(jd_ut, lat, lon)
        asc = houses_data['ascendant']
        mc = houses_data['midheaven']
        
        # Добавляем углы в список планет
        planets['Ascendant'] = {
            'longitude': asc,
            'sign': self.zodiac_signs[floor(asc / 30)],
            'position_in_sign': asc % 30,
            'sign_emoji': self.get_sign_emoji(asc)
        }
        planets['Midheaven'] = {
            'longitude': mc,
            'sign': self.zodiac_signs[floor(mc / 30)],
            'position_in_sign': mc % 30,
            'sign_emoji': self.get_sign_emoji(mc)
        }
        
        # Расчет аспектов
        aspects = self.calculate_aspects(planets, asc, mc)
        
        # Формирование результата
        return {
            'metadata': {
                'location': {
                    'city': city_name,
                    'coordinates': {
                        'latitude': lat,
                        'longitude': lon,
                        'elevation': elevation
                    },
                    'timezone': timezone_str
                },
                'datetime': {
                    'local': birth_local.strftime('%Y-%m-%d %H:%M:%S'),
                    'utc': birth_utc.strftime('%Y-%m-%d %H:%M:%S'),
                    'julian_day': jd_ut
                },
                'settings': {
                    'house_system': houses_data['system'],
                    'ephemeris': 'DE441'
                }
            },
            'planets': planets,
            'houses': houses_data['houses'],
            'aspects': aspects,
            'angles': {
                'ascendant': planets['Ascendant'],
                'midheaven': planets['Midheaven']
            }
        }

    def print_natal_chart(self, natal_chart):
        """Красивый вывод натальной карты"""
        print("\n" + "="*50)
        print("ТОЧНАЯ НАТАЛЬНАЯ КАРТА (профессиональный уровень)")
        print("="*50)
        
        # Основная информация
        meta = natal_chart['metadata']
        loc = meta['location']
        dt = meta['datetime']
        
        print(f"Место рождения: {loc['city']}")
        print(f"   Координаты: {loc['coordinates']['latitude']:.4f}°N, "
              f"{loc['coordinates']['longitude']:.4f}°E")
        print(f"   Высота: {loc['coordinates']['elevation']} м")
        print(f"   Часовой пояс: {loc['timezone']}")
        
        print(f"Время рождения:")
        print(f"   Местное: {dt['local']}")
        print(f"   UTC: {dt['utc']}")
        print(f"   Система домов: {meta['settings']['house_system']}")
        print(f"   Эфемериды: {meta['settings']['ephemeris']}")
        
        # Планеты
        print("Планеты и важные точки:")
        for name, data in natal_chart['planets'].items():
            retro = ' (R)' if data.get('retrograde', False) else ''
            print(f"   {name:10}: {data['position_in_sign']:6.2f}° {data['sign_emoji']} {data['sign']}{retro}")
        
        # Дома
        print("Дома гороскопа:")
        for i in range(1, 13):
            house = natal_chart['houses'][f'House_{i}']
            print(f"   Дом {i:2}: {house['position_in_sign']:6.2f}° {house['sign_emoji']} {house['sign']}")
        
        # Углы
        print("Важные углы:")
        asc = natal_chart['angles']['ascendant']
        mc = natal_chart['angles']['midheaven']
        print(f"   Асцендент : {asc['position_in_sign']:6.2f}° {asc['sign_emoji']} {asc['sign']}")
        print(f"   Середина Неба : {mc['position_in_sign']:6.2f}° {mc['sign_emoji']} {mc['sign']}")
        
        # Аспекты
        print("Основные аспекты:")
        for aspect in natal_chart['aspects'][:15]:  # Показываем топ-15 аспектов
            p1 = aspect['planet1']
            p2 = aspect['planet2']
            orb = aspect['orb']
            if orb < 0.5:
                orb_str = f"{orb:.3f}°"
            else:
                orb_str = f"{orb:.2f}°"
            
            print(f"   {p1:10} → {p2:10}: {aspect['aspect']:12} ({aspect['exact_angle']}°) "
                  f"орбис: {orb_str}")

    def save_natal_chart(self, natal_chart, filename='natal_chart.json'):
        """Сохранение карты в файл"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(natal_chart, f, ensure_ascii=False, indent=2)
        print(f"Натальная карта сохранена в файл: {filename}")

# Пример использования
if __name__ == "__main__":
    calculator = NatalChartCalculator()
    
    # Расчет карты для Москвы, 15 мая 1990, 13:30
    natal_chart = calculator.calculate_natal_chart(
        city_name="Москва",
        birth_datetime_local=datetime(1990, 5, 15, 13, 30),
        timezone_str="Europe/Moscow"
    )
    
    # Вывод результатов
    calculator.print_natal_chart(natal_chart)
    calculator.save_natal_chart(natal_chart)


ТОЧНАЯ НАТАЛЬНАЯ КАРТА (профессиональный уровень)
Место рождения: Москва
   Координаты: 55.6256°N, 37.6064°E
   Высота: 0.0 м
   Часовой пояс: Europe/Moscow
Время рождения:
   Местное: 1990-05-15 13:30:00
   UTC: 1990-05-15 09:30:00
   Система домов: Regiomontanus
   Эфемериды: DE441
Планеты и важные точки:
   Sun       :  24.30° ♉ Taurus
   Moon      :  25.88° ♑ Capricorn
   Mercury   :   8.03° ♉ Taurus (R)
   Venus     :  12.70° ♈ Aries
   Mars      :  18.26° ♓ Pisces
   Jupiter   :   9.52° ♋ Cancer
   Saturn    :  25.25° ♑ Capricorn (R)
   Uranus    :   9.19° ♑ Capricorn (R)
   Neptune   :  14.35° ♑ Capricorn (R)
   Pluto     :  16.17° ♏ Scorpio (R)
   Chiron    :  13.20° ♋ Cancer
   North Node:  10.26° ♒ Aquarius (R)
   South Node:   6.71° ♐ Sagittarius
   Ascendant :   5.37° ♍ Virgo
   Midheaven :  25.30° ♉ Taurus
Дома гороскопа:
   Дом  1:   5.37° ♍ Virgo
   Дом  2:  25.05° ♍ Virgo
   Дом  3:  18.96° ♎ Libra
   Дом  4:  25.30° ♏ Scorpio
   Дом  5:  10.19° ♑ Capricorn
   Дом  6: 

In [2]:
# Код для ИИ

import os
import pytz
from datetime import datetime
import swisseph as swe
import json
from math import floor
from typing import Dict, List, Tuple, Any
import logging

# Настройка логирования для отслеживания работы
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class MLNatalChartCalculator:
    def __init__(self):
        # Инициализация путей к эфемеридам
        current_dir = os.getcwd()
        ephe_path = os.path.join(current_dir, 'ephe')
        swe.set_ephe_path(ephe_path)
        swe.set_jpl_file('de441.eph')

        # ОПТИМИЗАЦИЯ: Упрощенные орбисы для ML (меньше шума)
        self.ORBS = {
            'conjunction': 8, 'opposition': 8, 'square': 8, 'trine': 8, 'sextile': 6,
            'quincunx': 3, 'semi-square': 3, 'semi-sextile': 3
        }

        # ОПТИМИЗАЦИЯ: Только основные планеты для ML (убраны астероиды)
        self.planets_ml = {
            swe.SUN: 'Sun',
            swe.MOON: 'Moon', 
            swe.MERCURY: 'Mercury',
            swe.VENUS: 'Venus',
            swe.MARS: 'Mars',
            swe.JUPITER: 'Jupiter',
            swe.SATURN: 'Saturn',
            swe.URANUS: 'Uranus',
            swe.NEPTUNE: 'Neptune',
            swe.PLUTO: 'Pluto',
            swe.TRUE_NODE: 'North_Node'  # ОПТИМИЗАЦИЯ: убраны южные узлы для уменьшения размерности
        }

        self.zodiac_signs = [
            "Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo",
            "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"
        ]

        # ОПТИМИЗАЦИЯ: Аспекты как константы для лучшей обработки ML
        self.aspects_ml = {
            0: ('conjunction', self.ORBS['conjunction']),
            60: ('sextile', self.ORBS['sextile']),
            90: ('square', self.ORBS['square']),
            120: ('trine', self.ORBS['trine']),
            180: ('opposition', self.ORBS['opposition'])
        }

    def get_city_coordinates(self, city_name: str) -> Tuple[float, float, float]:
        """Упрощенный геокодинг для ML"""
        # ОПТИМИЗАЦИЯ: Статический словарь городов (быстрее, надежнее)
        city_coordinates = {
            "москва": (55.7558, 37.6173, 156),
            "санкт-петербург": (59.9343, 30.3351, 3),
            "новосибирск": (55.0084, 82.9357, 177),
            "екатеринбург": (56.8389, 60.6057, 237),
            "нижний новгород": (56.3269, 44.0065, 200),
            "казань": (55.7964, 49.1089, 116),
            "челябинск": (55.1644, 61.4368, 228),
            "омск": (54.9924, 73.3686, 122),
            "самара": (53.1951, 50.1068, 156),
            "ростов-на-дону": (47.2224, 39.7185, 98),
            "уфа": (54.7351, 55.9587, 212),
            "красноярск": (56.0153, 92.8932, 287),
            "пермь": (58.0105, 56.2502, 171),
            "воронеж": (51.6720, 39.1843, 104),
            "волгоград": (48.7080, 44.5133, 158),
            "калининград": (54.7104, 20.4522, 25),
            "мга": (59.7569, 31.0609, 33)
        }
        
        city_lower = city_name.strip().lower()
        return city_coordinates.get(city_lower, (55.7558, 37.6173, 156))

    def calculate_planet_positions(self, jd_ut: float) -> Dict[str, Dict]:
        """Оптимизированный расчет для ML"""
        positions = {}
        for planet_id, name in self.planets_ml.items():
            try:
                flags = swe.FLG_SWIEPH | swe.FLG_SPEED
                pos, ret_flags = swe.calc_ut(jd_ut, planet_id, flags)
                
                lon = pos[0] % 360
                sign_index = floor(lon / 30)
                
                # ОПТИМИЗАЦИЯ: Только необходимые для ML данные
                positions[name] = {
                    'longitude': round(lon, 6),  # ОПТИМИЗАЦИЯ: Округление для уменьшения размера
                    'sign': self.zodiac_signs[sign_index],
                    'sign_index': sign_index,  # ОПТИМИЗАЦИЯ: Числовой индекс для ML
                    'position_in_sign': round(lon % 30, 4),
                    'retrograde': pos[3] < 0,
                    'speed': round(pos[3], 6)
                    # ОПТИМИЗАЦИЯ: Убраны emoji и декоративные поля
                }
            except Exception as e:
                logger.warning(f"Ошибка расчета для {name}: {e}")
                continue
                
        return positions

    def calculate_houses_ml(self, jd_ut: float, lat: float, lon: float) -> Dict:
        """Упрощенный расчет домов для ML"""
        try:
            # ОПТИМИЗАЦИЯ: Используем только Placidus для consistency
            hsys = b'P'
            cusps, ascmc = swe.houses(jd_ut, lat, lon, hsys)
            
            houses = {}
            for i, cusp in enumerate(cusps[:12]):
                cusp_deg = cusp % 360
                sign_index = floor(cusp_deg / 30)
                
                houses[i+1] = {  # ОПТИМИЗАЦИЯ: Числовые ключи вместо строк
                    'cusp_longitude': round(cusp_deg, 6),
                    'sign': self.zodiac_signs[sign_index],
                    'sign_index': sign_index,
                    'position_in_sign': round(cusp_deg % 30, 4)
                }
            
            return {
                'houses': houses,
                'ascendant': round(ascmc[0] % 360, 6),
                'midheaven': round(ascmc[1] % 360, 6),
                'house_system': 'Placidus'
            }
        except Exception as e:
            logger.error(f"Ошибка расчета домов: {e}")
            # Возвращаем значения по умолчанию
            return self._get_default_houses()

    def _get_default_houses(self) -> Dict:
        """Резервные значения домов при ошибке расчета"""
        houses = {}
        for i in range(12):
            houses[i+1] = {
                'cusp_longitude': round(i * 30.0, 6),
                'sign': self.zodiac_signs[i],
                'sign_index': i,
                'position_in_sign': 0.0
            }
        return {
            'houses': houses,
            'ascendant': 0.0,
            'midheaven': 0.0,
            'house_system': 'Placidus'
        }

    def calculate_aspects_ml(self, planets: Dict, asc: float, mc: float) -> List[Dict]:
        """Оптимизированный расчет аспектов для ML"""
        aspects = []
        
        # ОПТИМИЗАЦИЯ: Создаем объединенный словарь точек
        all_points = {**planets}
        all_points['Ascendant'] = {'longitude': asc}
        all_points['Midheaven'] = {'longitude': mc}
        
        point_names = list(all_points.keys())
        
        for i in range(len(point_names)):
            for j in range(i + 1, len(point_names)):
                p1, p2 = point_names[i], point_names[j]
                lon1, lon2 = all_points[p1]['longitude'], all_points[p2]['longitude']
                
                distance = abs(lon1 - lon2)
                angle = min(distance, 360 - distance)
                
                # Проверяем все аспекты
                for aspect_angle, (aspect_name, orb) in self.aspects_ml.items():
                    if abs(angle - aspect_angle) <= orb:
                        aspects.append({
                            'point1': p1,
                            'point2': p2,
                            'aspect': aspect_name,
                            'exact_angle': aspect_angle,
                            'actual_angle': round(angle, 4),
                            'orb': round(abs(angle - aspect_angle), 4),
                            'strength': 1.0 - (abs(angle - aspect_angle) / orb)  # ОПТИМИЗАЦИЯ: Сила аспекта для ML
                        })
                        break  # ОПТИМИЗАЦИЯ: Один аспект на пару
        
        # ОПТИМИЗАЦИЯ: Сортировка по силе аспекта
        aspects.sort(key=lambda x: x['strength'], reverse=True)
        return aspects

    def get_planet_house_placement(self, planets: Dict, houses: Dict) -> Dict:
        """Определение расположения планет в домах"""
        house_placement = {}
        
        for planet_name, planet_data in planets.items():
            planet_lon = planet_data['longitude']
            
            # Находим дом для планеты
            for house_num, house_data in houses.items():
                next_house_num = house_num + 1 if house_num < 12 else 1
                next_house_lon = houses[next_house_num]['cusp_longitude']
                
                # Корректировка для перехода через 360°
                current_lon = house_data['cusp_longitude']
                if next_house_lon < current_lon:
                    next_house_lon += 360
                    adjusted_planet_lon = planet_lon + 360 if planet_lon < current_lon else planet_lon
                else:
                    adjusted_planet_lon = planet_lon
                
                if current_lon <= adjusted_planet_lon < next_house_lon:
                    house_placement[planet_name] = house_num
                    break
            else:
                house_placement[planet_name] = 1  # По умолчанию в 1 дом
        
        return house_placement

    def calculate_natal_chart_ml(self, city_name: str, birth_datetime_local: datetime, timezone_str: str) -> Dict[str, Any]:
        """Основной метод для генерации ML-оптимизированной натальной карты"""
        try:
            # Получение координат
            lat, lon, elevation = self.get_city_coordinates(city_name)
            
            # Преобразование времени
            local_tz = pytz.timezone(timezone_str)
            birth_local = local_tz.localize(birth_datetime_local)
            birth_utc = birth_local.astimezone(pytz.utc)
            
            # Юлианская дата
            jd_ut = swe.julday(
                birth_utc.year,
                birth_utc.month, 
                birth_utc.day,
                birth_utc.hour + birth_utc.minute/60 + birth_utc.second/3600
            )
            
            # Расчет компонентов
            planets = self.calculate_planet_positions(jd_ut)
            houses_data = self.calculate_houses_ml(jd_ut, lat, lon)
            house_placement = self.get_planet_house_placement(planets, houses_data['houses'])
            aspects = self.calculate_aspects_ml(planets, houses_data['ascendant'], houses_data['midheaven'])
            
            # ОПТИМИЗАЦИЯ: ML-оптимизированная структура
            return {
                'metadata': {
                    'location': {
                        'city': city_name,
                        'lat': round(lat, 4),
                        'lon': round(lon, 4),
                        'elevation': round(elevation, 1)
                    },
                    'datetime': {
                        'local': birth_local.isoformat(),
                        'utc': birth_utc.isoformat(),
                        'jd': round(jd_ut, 6)
                    },
                    'calculation': {
                        'house_system': houses_data['house_system'],
                        'ephemeris': 'DE441'
                    }
                },
                
                # ОПТИМИЗАЦИЯ: Четкое разделение данных
                'planets': planets,
                'houses': houses_data['houses'],
                'angles': {
                    'ascendant': {
                        'longitude': houses_data['ascendant'],
                        'sign': self.zodiac_signs[floor(houses_data['ascendant'] / 30)],
                        'sign_index': floor(houses_data['ascendant'] / 30)
                    },
                    'midheaven': {
                        'longitude': houses_data['midheaven'],
                        'sign': self.zodiac_signs[floor(houses_data['midheaven'] / 30)],
                        'sign_index': floor(houses_data['midheaven'] / 30)
                    }
                },
                
                'placements': house_placement,  # ОПТИМИЗАЦИЯ: Отдельный объект для домов планет
                'aspects': aspects,
                
                # ОПТИМИЗАЦИЯ: ML-фичи для быстрого доступа
                'ml_features': {
                    'sign_distribution': self._get_sign_distribution(planets, houses_data),
                    'aspect_patterns': self._get_aspect_patterns(aspects),
                    'element_balance': self._get_element_balance(planets)
                }
            }
            
        except Exception as e:
            logger.error(f"Ошибка расчета натальной карты: {e}")
            raise

    def _get_sign_distribution(self, planets: Dict, houses_data: Dict) -> Dict[str, int]:
        """Распределение планет по знакам для ML-анализа"""
        distribution = {sign: 0 for sign in self.zodiac_signs}
        
        for planet_data in planets.values():
            distribution[planet_data['sign']] += 1
            
        return distribution

    def _get_aspect_patterns(self, aspects: List[Dict]) -> Dict[str, int]:
        """Паттерны аспектов для ML"""
        patterns = {
            'conjunctions': 0,
            'squares': 0, 
            'trines': 0,
            'oppositions': 0,
            'sextiles': 0
        }
        
        for aspect in aspects:
            if aspect['aspect'] in patterns:
                patterns[aspect['aspect']] += 1
                
        return patterns

    def _get_element_balance(self, planets: Dict) -> Dict[str, int]:
        """Баланс элементов для ML-анализа"""
        elements = {
            'fire': ['Aries', 'Leo', 'Sagittarius'],
            'earth': ['Taurus', 'Virgo', 'Capricorn'],
            'air': ['Gemini', 'Libra', 'Aquarius'], 
            'water': ['Cancer', 'Scorpio', 'Pisces']
        }
        
        balance = {element: 0 for element in elements}
        
        for planet_data in planets.values():
            for element, signs in elements.items():
                if planet_data['sign'] in signs:
                    balance[element] += 1
                    break
                    
        return balance

    def save_ml_chart(self, natal_chart: Dict, filename: str = 'natal_chart_ml.json') -> None:
        """Сохранение в ML-оптимизированном формате"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(natal_chart, f, ensure_ascii=False, indent=2, separators=(',', ':'))
        logger.info(f"ML-натальная карта сохранена: {filename}")

# Пример использования
if __name__ == "__main__":
    calculator = MLNatalChartCalculator()
    
    # Расчет оптимизированной карты
    natal_chart_ml = calculator.calculate_natal_chart_ml(
        city_name="мга",
        birth_datetime_local=datetime(1975, 8, 13, 20, 48),
        timezone_str="Europe/Moscow"
    )
    
    # Сохранение для ML-модели
    calculator.save_ml_chart(natal_chart_ml, 'natal_chart.json')
    
    # Вывод информации о структуре данных
    print("ML-Оптимизированная натальная карта готова!")
    print(f"Планет рассчитано: {len(natal_chart_ml['planets'])}")
    print(f"Аспектов найдено: {len(natal_chart_ml['aspects'])}")
    print(f"Размер файла: {len(json.dumps(natal_chart_ml))} байт")

INFO:__main__:ML-натальная карта сохранена: natal_chart.json


ML-Оптимизированная натальная карта готова!
Планет рассчитано: 11
Аспектов найдено: 24
Размер файла: 7504 байт


In [3]:
natal_chart_ml

{'metadata': {'location': {'city': 'мга',
   'lat': 59.7569,
   'lon': 31.0609,
   'elevation': 33},
  'datetime': {'local': '1975-08-13T20:48:00+03:00',
   'utc': '1975-08-13T17:48:00+00:00',
   'jd': 2442638.241667},
  'calculation': {'house_system': 'Placidus', 'ephemeris': 'DE441'}},
 'planets': {'Sun': {'longitude': 140.359236,
   'sign': 'Leo',
   'sign_index': 4,
   'position_in_sign': 20.3592,
   'retrograde': False,
   'speed': 0.9602},
  'Moon': {'longitude': 225.823684,
   'sign': 'Scorpio',
   'sign_index': 7,
   'position_in_sign': 15.8237,
   'retrograde': False,
   'speed': 13.695289},
  'Mercury': {'longitude': 152.712818,
   'sign': 'Virgo',
   'sign_index': 5,
   'position_in_sign': 2.7128,
   'retrograde': False,
   'speed': 1.81051},
  'Venus': {'longitude': 160.603333,
   'sign': 'Virgo',
   'sign_index': 5,
   'position_in_sign': 10.6033,
   'retrograde': True,
   'speed': -0.295798},
  'Mars': {'longitude': 59.30336,
   'sign': 'Taurus',
   'sign_index': 1,
   'p